In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
train1 = pd.read_csv('train_2.csv')

In [3]:
missing = train1.isnull()

In [4]:
train1 = train1.fillna(0)

In [5]:
train1.iloc[:,1:] = train1.iloc[:,1:].astype(np.float32)

In [6]:
n = train1.shape[0]

Check projection period and if all in contained in training set

In [ ]:
key = pd.read_csv('key_2.csv')
extract_page = key.Page.str.extract('(.+)_....-..-..')
extract_page = np.unique(extract_page)
set(train1.Page) == set(extract_page)
# True

In [10]:
key = pd.read_csv('key_2.csv')
len(set(key.Page.str.slice(-10)))

62

Extract Page information

In [21]:
a = train1.Page.str.split('mediawiki.org|wikimedia.org|wikipedia.org',expand=True)

In [22]:
b = np.where(a.loc[:,2].isnull(),a.loc[:,1],a.loc[:,2])

In [23]:
_,index_array = np.unique(b,return_inverse=True) # access & agent

In [24]:
website_array = [train1.Page.str.contains(web).values.astype(np.int8) \
                 for web in ['mediawiki.org','wikimedia.org','wikipedia.org']] # website

In [25]:
lang_array = train1.Page.str.extract('([a-z]{2}).wikipedia').fillna('nan').values.flatten()

In [26]:
_,lang_array = np.unique(lang_array,return_inverse=True) # language

In [28]:
Page_X = np.stack([np.arange(train1.shape[0]),lang_array,index_array] + website_array,1)

In [137]:
np.savetxt('Page_X',Page_X,fmt="%d",delimiter=",")

In [8]:
Page_X = np.loadtxt('Page_X',delimiter=",")

Extract date information

In [9]:
date_index=pd.to_datetime(train1.columns[1:]).to_series().reset_index(drop=True)

In [10]:
def extract_date_info(timestamp,mss):
    return np.array([mss, timestamp.weekday(), timestamp.month, timestamp.day])

Create training and test data for GBM

In [11]:
max_ = lambda df,start,interval: np.max(df.iloc[:,start-interval:start].values,1)
min_ = lambda df,start,interval: np.min(df.iloc[:,start-interval:start].values,1)
std_ = lambda df,start,interval: np.std(df.iloc[:,start-interval:start].values,1)
mean_ = lambda df,start,interval: np.mean(df.iloc[:,start-interval:start].values,1)
growth_ = lambda df,start,interval: np.mean(df.iloc[:,start-interval:start],1) -\
                                    np.mean(df.iloc[:,start-2*interval:start-interval],1)

In [12]:
fun_list = [max_,min_,std_,mean_,growth_]

In [13]:
interval_list = [14,30,60]

In [14]:
T = train1.shape[1]

Test dataset

In [230]:
j = T - 64

In [231]:
Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)

In [232]:
with open('testData','wb') as f:
    for i in range(j+2,j+64):
        temp = np.concatenate([train1.iloc[:,i].values[:,np.newaxis],\
                               Page_X,\
                               np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,4)),\
                               Lookback_X,\
                               train1.iloc[:,j-5:j].values],1).astype(np.float32)
        nonmissing_index = np.logical_not(missing.iloc[:,i].values)
        np.savetxt(f,temp[nonmissing_index],delimiter=',')

Training dataset

In [235]:
downSampleRate = 0.1

In [242]:
with open('trainData','wb') as f:
    for j in range(T-65*2,250,-80):
        Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)
        for i in range(j+2,j+64):
            temp = np.concatenate([train1.iloc[:,i].values[:,np.newaxis],\
                                   Page_X,\
                                   np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,4)),\
                                   Lookback_X,\
                                   train1.iloc[:,j-10:j].values],1).astype(np.float32)
            nonmissing_index = np.logical_not(missing.iloc[:,i].values)
            np.savetxt(f,temp[nonmissing_index][np.random.rand(nonmissing_index.sum())<downSampleRate],delimiter=',')

Save to binary lgbm.Dataset format

In [15]:
j = T - 64

In [16]:
Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)

In [17]:
chunks = 10

In [ ]:
for k in range(chunks):
    for i in range(j+2,j+64):
        y_ = train1.iloc[:,i].values
        temp = np.concatenate([Page_X,\
                               np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,4)),\
                               Lookback_X,\
                               train1.iloc[:,j-5:j].values],1).astype(np.float32)
        nonmissing_index = np.logical_not(missing.iloc[:,i].values)
        batch_ = nonmissing_index.sum()//chunks
        

In [18]:
10//3

3